In [519]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Flatten , Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model #모델 시각화 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard


tf.config.run_functions_eagerly(False)



ModuleNotFoundError: No module named 'django'

## 데이터 불러오기
- 앞서 크롤링하여 엑셀에 저장한 파일을 불러온다.

In [487]:
data_df = pd.read_excel('baseball_data.xlsx')
target_df = pd.read_excel('label.xlsx')
try:
    del data_df['Unnamed: 0']
    del target_df['Unnamed: 0']
except:
    pass


## 훈련세트, 검증세트, 테스트세트 분리
- 8:2비율로 훈련세트, 테스트 세트를 분리하고 또 다시 8:2로 훈련세트, 검증세트를 분리한다.
- 스코어를 예측하는 회귀모델을 위해 훈련세트, 검증세트, 테스트세트를 A팀과 B팀으로 분리한다.
- 레이블도 A팀 스코어 레이블, B팀 스코어 레이블, 승패 레이블로 분리한다.

In [488]:
#훈련세트, 검증세트, 테스트세트 분리 
X_full_set, X_test=  train_test_split(data_df, test_size = 0.2, random_state = 7)
y_full_set, y_test = train_test_split(target_df, test_size = 0.2, random_state = 7)
x_train, x_val, y_train, y_val = train_test_split(X_full_set,y_full_set, test_size=0.2, random_state=7)

#A B 나누기
A_train = x_train.iloc[:,:14]
B_train = x_train.iloc[:,14:]
A_valid = x_val.iloc[:,:14]   
B_valid = x_val.iloc[:,14:]
A_test = X_test.iloc[:,:14]
B_test = X_test.iloc[:,14:]

winlose_target_train = pd.DataFrame(y_train['승패'])
A_score_train = pd.DataFrame(y_train['A팀_스코어'])
B_score_train = pd.DataFrame(y_train['B팀_스코어'])

winlose_target_valid = pd.DataFrame(y_val['승패'])
A_score_valid = pd.DataFrame(y_val['A팀_스코어'])
B_score_valid = pd.DataFrame(y_val['B팀_스코어'])

winlose_target_test = pd.DataFrame(y_test['승패'])
A_score_test = pd.DataFrame(y_test['A팀_스코어'])
B_score_test = pd.DataFrame(y_test['B팀_스코어'])



## 데이터 전처리
- min-max 스케일링(정규화) 와 표준화 둘 중 더 좋은걸로 선택

In [489]:
#데이터 정규화
#scaler = StandardScaler() 
#mms = MinMaxScaler()

#x_train = scaler.fit_transform(x_train)
#x_val=scaler.fit_transform(x_val)
#X_test=scaler.transform(X_test)

In [490]:
#A_train= scaler.fit_transform(A_train)
#B_train= scaler.fit_transform(B_train)

#A_valid=scaler.fit_transform(A_valid)
#B_valid=scaler.fit_transform(B_valid)

#A_test=scaler.transform(A_test)
#B_test=scaler.transform(B_test)

In [491]:
#차원축소
#pca = PCA(n_components=7)
#X_train = pca.fit_transform(X_train)
#X_valid = pca.fit_transform(X_valid)


## 모델 만들기

In [492]:
#모델1 만들기
input = Input(shape=(14,),name='input')
hidden1 = Dense(16,activation='relu',name='hidden1')(input)
hidden2 = Dense(8,activation='relu',name='hidden2')(hidden1)
hidden3= Dense(4,activation='relu',name='hidden3')(hidden2)
output = Dense(1,name='output')(hidden3)


model_1 = Model(inputs=input,outputs=output)

#모델2 만들기
input= Input(shape=(28,),name='input')
hidden_1 = Dense(64,activation='relu',name='hidden_1')(input)
hidden_2 = Dense(32,activation='relu',name='hidden_2')(hidden_1)
hidden_3 = Dense(16,activation='relu',name='hidden_3')(hidden_2)
hidden_4 = Dense(8,activation='relu',name='hidden_4')(hidden_3)
output = Dense(1,activation='sigmoid',name='output')(hidden_4)

model_2 = Model(inputs=input,outputs=output)

## 모델컴파일
- 회귀문제 : MSE(loss) , MAE(eval)

In [493]:
model_1.compile(loss='mse',
              optimizer=Adam(learning_rate=1e-2),metrics=['mae'])

model_2.compile(loss='binary_crossentropy', optimizer='Adam',metrics=['accuracy'])

## 모델학습

In [494]:
#A데이터로 학습
history = model_1.fit(A_train,
                    A_score_train,
                    epochs=50, validation_data=(A_valid,A_score_valid))

Epoch 1/50
39/39 [==============================] - 1s 5ms/step - loss: 10.3894 - mae: 2.4722 - val_loss: 5.8245 - val_mae: 1.8918
Epoch 2/50
39/39 [==============================] - 0s 3ms/step - loss: 4.5773 - mae: 1.5873 - val_loss: 4.5990 - val_mae: 1.7582
Epoch 3/50
39/39 [==============================] - 0s 2ms/step - loss: 3.3056 - mae: 1.3719 - val_loss: 3.5205 - val_mae: 1.5270
Epoch 4/50
39/39 [==============================] - 0s 2ms/step - loss: 3.1118 - mae: 1.3370 - val_loss: 3.1509 - val_mae: 1.3504
Epoch 5/50
39/39 [==============================] - 0s 2ms/step - loss: 2.8986 - mae: 1.3025 - val_loss: 3.0717 - val_mae: 1.3983
Epoch 6/50
39/39 [==============================] - 0s 2ms/step - loss: 3.2810 - mae: 1.4138 - val_loss: 3.1201 - val_mae: 1.3469
Epoch 7/50
39/39 [==============================] - 0s 2ms/step - loss: 2.8990 - mae: 1.3222 - val_loss: 2.4409 - val_mae: 1.2040
Epoch 8/50
39/39 [==============================] - 0s 2ms/step - loss: 2.6856 - mae: 1.2

In [495]:
#B데이터로 학습
#history = model_1.fit(B_train,
#                    B_score_train,
#                    epochs=50, validation_data=(B_valid,B_score_valid))

In [496]:
history = model_2.fit(x_train,winlose_target_train, epochs=50, validation_data=(x_val,winlose_target_valid))

Epoch 1/50
39/39 [==============================] - 1s 6ms/step - loss: 1.0121 - accuracy: 0.5089 - val_loss: 0.6465 - val_accuracy: 0.6494
Epoch 2/50
39/39 [==============================] - 0s 2ms/step - loss: 0.5864 - accuracy: 0.6851 - val_loss: 0.5391 - val_accuracy: 0.7240
Epoch 3/50
39/39 [==============================] - 0s 2ms/step - loss: 0.4942 - accuracy: 0.7573 - val_loss: 0.4814 - val_accuracy: 0.7695
Epoch 4/50
39/39 [==============================] - 0s 2ms/step - loss: 0.4359 - accuracy: 0.7930 - val_loss: 0.4367 - val_accuracy: 0.7922
Epoch 5/50
39/39 [==============================] - 0s 2ms/step - loss: 0.4066 - accuracy: 0.8198 - val_loss: 0.3708 - val_accuracy: 0.8409
Epoch 6/50
39/39 [==============================] - 0s 2ms/step - loss: 0.3607 - accuracy: 0.8417 - val_loss: 0.3391 - val_accuracy: 0.8506
Epoch 7/50
39/39 [==============================] - 0s 2ms/step - loss: 0.3359 - accuracy: 0.8555 - val_loss: 0.3155 - val_accuracy: 0.8701
Epoch 8/50
39/39 [==

## 모델 평가

In [497]:
model_1.evaluate(A_test,A_score_test)
model_2.evaluate(X_test,winlose_target_test)

13/13 [==============================] - 0s 1ms/step - loss: 0.2798 - accuracy: 0.8961


[0.27980315685272217, 0.8961039185523987]

## 예측
- 학습하지 않은 22년 10월 8일 3경기의 데이터를 예측하고자 한다.
- 단 이때 22년 10월 8일의 경기가 진행되지 않음을 가정하고 예측을 진행한다.
- 때문에 모델에 input으로 들어가는 데이터는 각 구단의 직전 3경기의 평균값을 입력하고 실제 22년 10월 8일의 데이터와 비교한다.
- 선발투수는 경기전 확정된 선발투수의 데이터로 입력값을 넣는다(직전 3경기 평균값X 당일 데이터O)
|22-10-8|A구단 점수|B구단 점수|
|------|----------|----------|
|SSG(A) : 삼성(B)|1|6|
|키움 : 두산|5|1|
|한화 : NC|5|6|


In [515]:
#예측1 SSG:삼성
SSG =[36.3,33.6,6.6,1,2.3,0,9.6,10,7.6,0.196,0.153,18,2.49,1.16]
samsung = [41.6,38.6,12.6,0.3,2.3,0,8,8.6,9.3,0.325,0.252,19,6.00,1.52]
dusan = [34.6,32.6,9.3,1,1.3,0,6.3,8,9,0.282,0.213,11.6,3.60,1.33]
kiwum = [50,44,12,1,5.3,0.6,8.6,11.6,11.6,0.268,0.213,21.6,2.11,0.95]
score_SSG_samsung =model_1.predict([SSG,samsung])
score_dusan_kiwum =model_1.predict([dusan,kiwum])
winlose_SSG_samsung= model_2.predict([[36.3,33.6,6.6,1,2.3,0,9.6,10,7.6,0.196,0.153,18,2.49,1.16,41.6,38.6,12.6,0.3,2.3,0,8,8.6,9.3,0.325,0.252,19,6.00,1.52]])
winlose_dusan_kiwum = model_2.predict([[34.6,32.6,9.3,1,1.3,0,6.3,8,9,0.282,0.213,11.6,3.60,1.33,50,44,12,1,5.3,0.6,8.6,11.6,11.6,0.268,0.213,21.6,2.11,0.95]])
print(f"SSG 예측점수:{score_SSG_samsung[0]}")
print(f"삼성 예측점수:{score_SSG_samsung[1]}")
print(f"두산 예측점수:{score_dusan_kiwum[0]}")
print(f"키움 예측점수:{score_dusan_kiwum[1]}")

print(f"SSG 우승확률 : {float((1-winlose_SSG_samsung)*100)}%")
print(f"삼성 우승확률 : {float((winlose_SSG_samsung)*100)}%")
print(f"두산 우승확률 : {float((1-winlose_dusan_kiwum)*100)}%")
print(f"키움 우승확률 : {float((winlose_dusan_kiwum)*100)}%")
print('================================')
print(winlose_dusan_kiwum)
print(winlose_SSG_samsung)


1/1 [==============================] - 0s 39ms/step
SSG 예측점수:[1.9077125]
삼성 예측점수:[5.8765464]
두산 예측점수:[3.981299]
키움 예측점수:[7.3295355]
SSG 우승확률 : 6.084018707275391%
삼성 우승확률 : 93.91597747802734%
두산 우승확률 : 81.46593475341797%
키움 우승확률 : 18.5340633392334%
[[0.18534063]]
[[0.9391598]]


## 모델 저장과 복원

In [518]:
#모델 저장
model_1.save('score_predict_model.h5')
model_2.save('win_lose_predict_model.h5')

#모델 복원
loaded_model_1 = models.load_model('score_predict_model.h5')
loaded_model_2 = models.load_model('win_lose_predict_model.h5')

NameError: name 'models' is not defined